In [3]:
import pandas as pd
import numpy as np
from functools import reduce
import os

# Basic knowledge: 
To begin this project, it would be good to hold a minimum understanding of the subject we will be analyzing. As I did not know much about this topic at the day the project started, I have recurred to the shark-attack wiki: https://en.wikipedia.org/wiki/Shark_attack



# Defining the dataset path, and importing it to begin basic dataset exploration

Questions: 
- how to add the dataset to gitignore?

In [21]:
# To follow along and access the DataSet, download it from KAGGLE using this link
# https://www.kaggle.com/teajay/global-shark-attacks

# Once you have downloaded the DataSet, change the dataset variable to match the 
# path where you have saved the 'attacks.csv' file.
dataset = 'attacks.csv' 
df = pd.read_csv(dataset, encoding='latin-1')

Now, we will check some basic information about the dataset, in order to formulate a more educated hypothesis which we could actually put to test with the data available.

In [22]:
display(df.shape) # To know the shape of the DF
display(df.columns) # To know which are the columns in the DF
display(df.count()) # To know how much data are we missin on each column

(25723, 24)

Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species ', 'Investigator or Source', 'pdf', 'href formula', 'href',
       'Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22',
       'Unnamed: 23'],
      dtype='object')

Case Number               8702
Date                      6302
Year                      6300
Type                      6298
Country                   6252
Area                      5847
Location                  5762
Activity                  5758
Name                      6092
Sex                       5737
Age                       3471
Injury                    6274
Fatal (Y/N)               5763
Time                      2948
Species                   3464
Investigator or Source    6285
pdf                       6302
href formula              6301
href                      6302
Case Number.1             6302
Case Number.2             6302
original order            6309
Unnamed: 22                  1
Unnamed: 23                  2
dtype: int64

In [29]:
df[["Fatal (Y/N)"]].describe()

,Fatal (Y/N)
count,5763
unique,8
top,N
freq,4293
